<a href="https://colab.research.google.com/github/MohamedBechir-prog/Optimisation-Projects/blob/main/Unit_Commitement/Unit_Commitement.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Unit Commitement Problem

In this notebook, I will incrementally implement a unit commitment problem using Gurobi. The goal is to meet the projected electricity demand at each time interval by optimally dispatching various types of power plants.

The time horizon spans a single day, divided into five periods to reflect typical fluctuations in electrical load throughout the day.

| Time Period | Demand (MW) |
| --- | --- |
| 12 pm to 6 am | 15000 |
| 6 am to 9 am | 30000 |
| 9 am to 3 pm | 25000 |
| 3 pm to 6 pm | 40000 |
| 6 pm to 12 pm | 27000 |

The dataset includes three types of conventional power plants, each characterized by its minimum and maximum power output, the number of units available, and generation costs.


| Type | Available Units | Minimum output (MW) | Maximum output (MW) | Generation cost (€/MWh) |
| --- | --- | --- | --- | --- |
| 1 | 12 |  850 | 2000 |1.5 |
| 2 | 10 | 1250 | 1750 |1.38 |
| 3 | 5 | 1500 | 4000 |2.35 |

The complexity of the model will gradually increase as the script progresses.

This project is part of the Optimization coursework from my master's program in Energy Systems Optimization at Mines Paris.

The example can also be found in the fifth edition of **Model Building in Mathematical Programming by H. Paul Williams on pages 271-272 and 326-327**.

## Basic Case

In [1]:
!pip install gurobipy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.3/14.3 MB 66.7 MB/s eta 0:00:00


In [2]:
import gurobipy as gp

In [3]:
data = {1 : [12,850,2000,1.5], 2 : [10,1250,1750,1.38], 3 : [5,1500,4000,2.75]}
costs = {1 : data[1][3], 2 : data[2][3], 3 : data[3][3]}
Pmin = {1 : data[1][1], 2 : data[2][1], 3 : data[3][1]}
Pmax = {1 : data[1][2], 2 : data[2][2], 3 : data[3][2]}
N = {1 : data[1][0], 2 : data[2][0], 3 : data[3][0]}
load = [15000,30000,25000,40000,27000]
periods = [i for i in range(1,6)]
types = [i for i in range(1,4)]
duration = [6,3,6,3,6]

## Mathematical Formulation of the Unit Commitment Problem 1.1

We define the following indices:

- $i \in \mathcal{I}$: set of power plant types  
- $t \in \mathcal{T}$: set of time periods

### 🔧 Decision Variables

- $Y_{i,t} \in \mathbb{N}$ : number of units of type $i$ operating at time $t$  
- $P_{i,t} \in \mathbb{R}_{\geq 0}$: power output of type $i$ at time $t$

### 📌 Parameters

- $\text{load}_t$: electricity demand at time $t$  
- $P^{\text{max}}_i$: maximum power output per unit of type $i$  
- $P^{\text{min}}_i$: minimum power output per unit of type $i$  
- $N_i$: number of available units of type $i$  
- $c_i$: cost per unit of power for type $i$  
- $d_t$: duration of time period $t$

### ✅ Constraints

1. **Demand Satisfaction**  
   $\sum_{i \in \mathcal{I}} P_{i,t} \geq \text{load}_t \quad \forall t \in \mathcal{T}$

2. **Power Output Bounds**  
   $P_{i,t} \leq P^{\text{max}}_i \cdot Y_{i,t} \quad \forall i \in \mathcal{I}, \forall t \in \mathcal{T}$  
   $P_{i,t} \geq P^{\text{min}}_i \cdot Y_{i,t} \quad \forall i \in \mathcal{I}, \forall t \in \mathcal{T}$

3. **Unit Availability**  
   $Y_{i,t} \leq N_i \quad \forall i \in \mathcal{I}, \forall t \in \mathcal{T}$

### 🎯 Objective Function

Minimize total generation cost:  
$\min \sum_{t \in \mathcal{T}} \sum_{i \in \mathcal{I}} c_i \cdot P_{i,t} \cdot d_t$

In [4]:
model = gp.Model()
# Creating variables
indices = [(i,t) for i in types for t in periods]
Y = model.addVars(indices,lb =0, vtype=gp.GRB.INTEGER,name=[f'Y{i}{t}' for i,t in indices])
P = model.addVars(indices,vtype=gp.GRB.CONTINUOUS,name=[f'P{t}' for t in indices])

# Constraints
for t in periods:
  model.addConstr(gp.quicksum(P[i,t] for i in types) >= load[t-1], name=f'Demand_{t}')

for i in types:
  for t in periods:
    model.addConstr(P[i,t] <= Pmax[i]*Y[i,t], name=f'Puissance_Max_{i},{t}')
    model.addConstr(P[i,t] >= Pmin[i]*Y[i,t], name=f'Puissance_Min_{i},{t}')
    model.addConstr(Y[i,t] <= N[i], name=f'Available_Power_Plant_{i},{t}')

# Objective function
model.setObjective(gp.quicksum(costs[i]*P[i,t]*duration[t-1] for i,t in indices),gp.GRB.MINIMIZE)

# Optimisation
model.optimize()


Restricted license - for non-production use only - expires 2026-11-23
Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (linux64 - "Ubuntu 22.04.4 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 50 rows, 30 columns and 90 nonzeros
Model fingerprint: 0x37fef50e
Variable types: 15 continuous, 15 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+03]
  Objective range  [4e+00, 2e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e+00, 4e+04]
Presolve removed 46 rows and 24 columns
Presolve time: 0.00s
Presolved: 4 rows, 6 columns, 12 nonzeros
Variable types: 0 continuous, 6 integer (0 binary)
Found heuristic solution: objective 958050.00000
Found heuristic solution: objective 947700.00000
Found heuristic solution: objective 933300.00000

Root relaxation: objective 8.694000e+05, 2 iterations, 0.00 seconds (0.00 work units)

    N

In [5]:
for v in model.getVars():
    print('%s %g' % (v.VarName, v.X))

Y11 -0
Y12 12
Y13 4
Y14 12
Y15 5
Y21 9
Y22 10
Y23 10
Y24 10
Y25 10
Y31 -0
Y32 -0
Y33 -0
Y34 0
Y35 -0
P(1, 1) 0
P(1, 2) 12500
P(1, 3) 7500
P(1, 4) 22500
P(1, 5) 9500
P(2, 1) 15000
P(2, 2) 17500
P(2, 3) 17500
P(2, 4) 17500
P(2, 5) 17500
P(3, 1) 0
P(3, 2) 0
P(3, 3) 0
P(3, 4) 0
P(3, 5) 0


## Unit Commitment Problem 1.2


In this formulation, we introduce fixed costs associated with basic power generation, representing the cost of maintaining minimum output levels. Additionally, we revise the generation cost structure: variable costs are now applied only to the portion of power produced above the minimum threshold.

- Fixed generation costs: $C^{\text{base}}_i$ applied per unit committed ($Y_{i,t}$)
- Variable generation costs: $c_i$ applied only to power produced above the minimum output

| Type | Cost per hour (€/h) | Cost in €/MWh above minimum | Startup cost (€)|
| --- | --- | --- | --- |
| 1 | 1000 | 2.00 | 2000 |
| 2 | 2600 | 1.30 | 1000 |
| 3 | 3000 | 3.00 | 500 |

The revised objective function becomes:

$\min \sum_{t \in \mathcal{T}} \sum_{i \in \mathcal{I}} \left[ c_i \cdot P_{i,t} + C^{\text{base}}_i \cdot Y_{i,t} - c_i \cdot P^{\text{min}}_i \cdot Y_{i,t} \right] \cdot d_t$

In [6]:
# Modification of input data
data = {1 : [12,850,2000,2,1000,2000], 2 : [10,1250,1750,1.3,2600,1000], 3 : [5,1500,4000,3,3000,500]}
types, N, Pmin, Pmax, costs, Cbase, Cstart  = gp.multidict(data)

Newobj = gp.quicksum((costs[i]*P[i,t]+ Cbase[i]*Y[i,t] - costs[i]*Pmin[i]*Y[i,t])*duration[t-1] for i,t in indices)
model.setObjective(Newobj,gp.GRB.MINIMIZE)
model.optimize()

Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (linux64 - "Ubuntu 22.04.4 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 50 rows, 30 columns and 90 nonzeros
Model fingerprint: 0x367d385d
Variable types: 15 continuous, 15 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+03]
  Objective range  [4e+00, 9e+03]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e+00, 4e+04]

MIP start from previous solve produced solution with objective 1.08045e+06 (0.04s)
Loaded MIP start from previous solve with objective 1.08045e+06

Presolve removed 46 rows and 24 columns
Presolve time: 0.01s
Presolved: 4 rows, 6 columns, 12 nonzeros
Found heuristic solution: objective 1006800.0000
Variable types: 0 continuous, 6 integer (0 binary)

Root relaxation: objective 9.780000e+05, 1 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Curren

### Introducing Starting up costs

In order to incorporate startup costs, we need to introduce a new **positive** variable $d_{it}$ that accounts for the number of starting units of type $i$ at period $t$.

For the first period : $d_{i1} = Y_{i1} \quad \forall i \in \mathcal{I}$

For the other periods, it is defined as the difference between the actual number of activated plants and that in the previous period : $ {d_{it}\geq Y_{it} - Y_{it-1}}  \quad \forall i \in \mathcal{I} \quad \forall t {\geq 0}$

With this formulation, $d_{it}$ will always be equal to the difference of the number of plants between two periods. If the model decides to reduce the number of activated plants, then $d_{it}$ would be equal to 0.

In [7]:
# We define d as the difference between the current number of activated PP and that of the previous period
# For the first period, it will be equal to Yit
# d is an integer number, always positive because we don't consider the case where we desactivate PPs

d = model.addVars(indices,lb=0,vtype=gp.GRB.INTEGER,name=[f'd{i}{t}' for i,t in indices])
for i in types:
  for t in periods:
    if t == 1:
      model.addConstr(d[i,t] == Y[i,t], name=f'Activated_PP_{i},{t}')
    else:
      model.addConstr(d[i,t] >= Y[i,t] - Y[i,t-1], name=f'Activated_PP_{i},{t}')
obj = model.getObjective()
model.setObjective(obj + gp.quicksum(Cstart[i]*d[i,t] for i,t in indices),gp.GRB.MINIMIZE)
model.optimize()

Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (linux64 - "Ubuntu 22.04.4 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 65 rows, 45 columns and 132 nonzeros
Model fingerprint: 0x273e2a99
Variable types: 15 continuous, 30 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+03]
  Objective range  [4e+00, 9e+03]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e+00, 4e+04]

MIP start from previous solve produced solution with objective 1.0149e+06 (0.02s)
Loaded MIP start from previous solve with objective 1.0149e+06

Presolve removed 33 rows and 3 columns
Presolve time: 0.00s
Presolved: 32 rows, 42 columns, 96 nonzeros
Variable types: 0 continuous, 42 integer (0 binary)

Root relaxation: objective 1.011257e+06, 10 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Wor

### Reserve Constraint

Generators must meet predicted demand, but they must also have sufficient reserve capacity to be able to cope with the situation where actual demand exceeds predicted demand.

For each time period $t \in \mathcal{T}$, the constraint is :

$\sum_{i \in \mathcal{I}} Y_{i,t} \cdot P^{\text{max}}_i \geq (1 + r) \cdot \text{load}_t$

For this model, the set of selected thermal generators must be able to produce as much as 115% of predicted demand.

In [8]:
# Reserve is 15%
r = 0.15

# Adding reserve constraint
for t in periods:
  model.addConstr(gp.quicksum(Y[i,t]*Pmax[i] for i in types) >= (1+r)*load[t-1], name=f'Reserve_{t}')

model.update()
model.optimize()

Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (linux64 - "Ubuntu 22.04.4 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 70 rows, 45 columns and 147 nonzeros
Model fingerprint: 0x5339dc00
Variable types: 15 continuous, 30 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+03]
  Objective range  [4e+00, 9e+03]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e+00, 5e+04]

MIP start from previous solve did not produce a new incumbent solution
MIP start from previous solve violates constraint Reserve_4 by 4500.000000000

Presolve removed 33 rows and 3 columns
Presolve time: 0.00s
Presolved: 37 rows, 42 columns, 111 nonzeros
Variable types: 0 continuous, 42 integer (0 binary)
Found heuristic solution: objective 1356800.0000

Root relaxation: objective 1.012257e+06, 12 iterations, 0.00 seconds (0.00 work units)

    Nodes    |   

### Planification cyclique

In this section, we assume that the daily electricity demand pattern repeats identically from one day to the next. To reflect this, we introduce a **cyclic** option that links the last time period of the current day to the first period of the next day.

This cyclic constraint ensures continuity in unit commitment decisions, allowing the model to anticipate the operational state for the following day. It is particularly useful for planning startup/shutdown sequences and maintaining reserve margins across day boundaries.

The only change in the model formulation is in the definition of $d_{it}$ where we replaced $Y_{it-1}$ by $Y_{i,prev(t)}$ if the cyclic option is selected.

In [9]:
# A funtion that returns the previous period based on the selected option
def prev(period, option='cyclique'):
  if option == 'cyclique':
    if period == 1:
      return 5
  return period-1

In [10]:
option = 'cyclique'

model = gp.Model()

# Creating variables
indices = [(i,t) for i in types for t in periods]
Y = model.addVars(indices,lb =0, vtype=gp.GRB.INTEGER,name=[f'Y{i}{t}' for i,t in indices])
P = model.addVars(indices,vtype=gp.GRB.CONTINUOUS,name=[f'P{t}' for t in indices])

# Constraints
for t in periods:
  model.addConstr(gp.quicksum(P[i,t] for i in types) >= load[t-1], name=f'Demand_{t}')

for i in types:
  for t in periods:
    model.addConstr(P[i,t] <= Pmax[i]*Y[i,t], name=f'Puissance_Max_{i},{t}')
    model.addConstr(P[i,t] >= Pmin[i]*Y[i,t], name=f'Puissance_Min_{i},{t}')
    model.addConstr(Y[i,t] <= N[i], name=f'Available_Power_Plant_{i},{t}')

d = model.addVars(indices,lb=0,vtype=gp.GRB.INTEGER,name=[f'd{i}{t}' for i,t in indices])
for i in types:
  for t in periods:
    if t == 1 and option != 'cyclique':
      model.addConstr(d[i,t] == Y[i,t], name=f'Activated_PP_{i},{t}')
    else:
      model.addConstr(d[i,t] >= Y[i,t] - Y[i,prev(t,option)], name=f'Activated_PP_{i},{t}')

r = 0.15
for t in periods:
  model.addConstr(gp.quicksum(Y[i,t]*Pmax[i] for i in types) >= (1+r)*load[t-1], name=f'Reserve_{t}')

# Objective
Newobj = gp.quicksum((costs[i]*P[i,t]+ Cbase[i]*Y[i,t] - costs[i]*Pmin[i]*Y[i,t])*duration[t-1]+Cstart[i]*d[i,t] for i,t in indices)
model.setObjective(Newobj,gp.GRB.MINIMIZE)

model.optimize()

Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (linux64 - "Ubuntu 22.04.4 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 70 rows, 45 columns and 150 nonzeros
Model fingerprint: 0xc3eb587b
Variable types: 15 continuous, 30 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+03]
  Objective range  [4e+00, 9e+03]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e+00, 5e+04]
Presolve removed 30 rows and 0 columns
Presolve time: 0.00s
Presolved: 40 rows, 45 columns, 120 nonzeros
Variable types: 0 continuous, 45 integer (0 binary)
Found heuristic solution: objective 1320300.0000

Root relaxation: objective 9.855143e+05, 12 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 9855

## Incorporating Hydropower Plants : Model 3.1

In addition to thermal units, two hydroelectric generators are available. Each produces a fixed amount of power when activated:

| Hydro Plant | Output (MW) |
|-------------|-------------|
| A           | 900         |
| B           | 1400        |

Unlike thermal generators, hydroelectric units have a distinct cost structure:

- A small hourly operating cost
- A startup cost incurred when the unit is turned on
- A reservoir depletion cost, reflecting the reduction in water level over time

| Hydro Plant | Hourly Cost (€/hr) | Startup Cost (€) | Reservoir Depletion Rate (m/hr) |
|-------------|-------------|--------------|---------------------------|
| A           | 90         | 1500        | 0.31                      |
| B           | 150        | 1200        | 0.47                      |

To maintain sustainability, the reservoir must be fully replenished by the end of the time horizon. Replenishment is achieved by pumping water back into the reservoir, which consumes electricity at a rate of **3000 MWh per meter of elevation**.

#### **Reserve Requirement**

Generators must not only meet the forecasted demand but also maintain sufficient reserve capacity to handle unexpected surges. In this model, the combined capacity of selected thermal and hydro units must be at least **115% of the predicted demand** at each time period.

#### **Decision Variables**

- $H_{h,t} \in \{0,1\}$: Binary variable indicating whether hydro unit $h$ is on at time $t$.
- $b_{h,t} \in \{0,1\}$: Binary variable indicating whether hydro unit $h$ is **activated** at time $t$ (i.e., turned on).
- $ \text{Pumping}_t \in \mathbb{R}_{\geq 0} $: Amount of electricity used to pump water into the reservoir during time period $t$
- $ \text{Height}_t \in \mathbb{R}_{\geq 0} $: Water level (in meters) of the reservoir at time period $t$

#### **Startup of Hydroelectric Plants**

To track hydro unit startups, we define:

$b_{h,t} \geq H_{h,t} - H_{h,t-1}$

This constraint ensures that $b_{h,t} = 1$ only when a unit transitions from off to on.

#### **Updated Demand and Reserve Constraints**

We remove the original demand and reserve constraints and replace them with updated versions that include hydro units $\forall t \in \mathcal{T}$, while accounting for the cost (in terms of energy) that is proportional to the amount of energy required to raise the water level.  :

- **Demand Satisfaction**  
  $\sum_{i \in \mathcal{I}} P_{i,t} + \sum_{h \in \mathcal{H}} H_{h,t} \cdot P^{\text{hyd}}_h \geq \text{load}_t + pumping_t$

- **Reserve Requirement**  
  $\sum_{i \in \mathcal{I}} Y_{i,t} \cdot P^{\text{max}}_i + \sum_{h \in \mathcal{H}} (1 - H_{h,t}) \cdot P^{\text{hyd}}_h \geq (1 + r) \cdot \text{load}_t$

Note: The hydro units contribute to reserve **only when they are off**, since they can be quickly activated.

#### **Updated Objective Function**

We augment the original objective function to include:

- Hourly operating costs for hydro units: $C^{\text{hyd}}_h$
- Startup costs for hydro units: $\text{StartHyd}_h$

The new objective becomes:

$\min \left[ \text{Original Objective} + \sum_{t \in \mathcal{T}} \sum_{h \in \mathcal{H}} \left( C^{\text{hyd}}_h \cdot H_{h,t} \cdot d_t + \text{StartHyd}_h \cdot b_{h,t} \right) \right]$

#### **Reservoir Balance Constraint**

To track the evolution of the reservoir level over time, we define a **reservoir height balance** equation for each time period $t \in \mathcal{T}$:

$\text{Height}_t = \text{Height}_{prev(t)} + \dfrac{d_t \cdot \text{Pumping}_t}{3000} - \sum_{h \in \mathcal{H}} H_{h,t} \cdot d_t \cdot v_h$

With $v_h$ being the velocity of water level reduction for Hydropower plant $h$ (m/h).

**Note:** If the **cyclic** mode is not enabled, the expression $prev(t)$ should be replaced with $t - 1$ for all $t \geq 2$. Additionally, a separate constraint must be introduced to handle the transition between the last and first periods, ensuring continuity across the time horizon.

In [11]:
hydro, Phyd, vitesse,StartHyd,Chyd = gp.multidict({1: [900,0.31,1500,90], 2:[1400,0.47, 1200, 150]})

In [15]:
indices_hydro = [(h,t) for h in hydro for t in periods]

H = model.addVars(indices_hydro, vtype=gp.GRB.BINARY,name=[f'H{h}{t}' for h,t in indices_hydro])
b = model.addVars(indices_hydro, vtype=gp.GRB.BINARY,name=[f'b{h}{t}' for h,t in indices_hydro])
pumping = model.addVars(periods, lb = 0, vtype=gp.GRB.CONTINUOUS,name=[f'Pumping{t}' for t in periods])
height = model.addVars(periods, lb = 0, vtype=gp.GRB.CONTINUOUS,name=[f'Reservoir height{t}' for t in periods])

for t in periods:
  for h in hydro:
    model.addConstr(b[h,t] >= H[h,t] - H[h,prev(t,option)], name=f'Activated_Hydro_{h},{t}')
  model.remove(model.getConstrByName(f'Reserve_{t}'))
  model.remove(model.getConstrByName(f'Demand_{t}'))
  model.addConstr(gp.quicksum(P[i,t] for i in types) + gp.quicksum((H[h,t])*Phyd[h] for h in hydro) >= load[t-1] + pumping[t] , name=f'Demand_{t}')
  model.addConstr(gp.quicksum(Y[i,t]*Pmax[i] for i in types) + gp.quicksum((1-H[h,t])*Phyd[h] for h in hydro) >= (1+r)*load[t-1], name=f'Reserve_{t}')
  model.addConstr(height[t] == height[prev(t)] + duration[t-1]*pumping[t]/3000 - gp.quicksum(H[h,t]*duration[t-1]*vitesse[h] for h in hydro))

obj = model.getObjective()
model.setObjective(obj + gp.quicksum(Chyd[h]*H[h,t]*duration[t-1] + StartHyd[h]*b[h,t] for h,t in indices_hydro),gp.GRB.MINIMIZE)

model.update()
model.optimize()

Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (linux64 - "Ubuntu 22.04.4 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 85 rows, 115 columns and 230 nonzeros
Model fingerprint: 0x86a29874
Variable types: 25 continuous, 90 integer (60 binary)
Coefficient statistics:
  Matrix range     [1e-03, 4e+03]
  Objective range  [4e+00, 9e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [5e+00, 4e+04]

MIP start from previous solve produced solution with objective 990730 (0.01s)
MIP start from previous solve produced solution with objective 989890 (0.01s)
MIP start from previous solve produced solution with objective 988540 (0.02s)
MIP start from previous solve produced solution with objective 988360 (0.02s)
Loaded MIP start from previous solve with objective 988360

Presolve removed 30 rows and 40 columns
Presolve time: 0.00s
Presolved: 55 rows, 75 colu

### 5.3

In [ ]:
paliers = {1 : {(1,1) : [900, 0.31, 90], (1,2) : [950, 0.33, 95], (1,3): [1000, 0.35, 105], (1,4): [1100, 0.38, 120]},
2 : {(2,1) : [1400, 0.47, 150], (2,2) : [1500, 0.5, 165], (2,3): [1600, 0.53, 185], (2,4): [1700, 0.56, 210]} }

In [ ]:
keys, Phyd, vitesse,Chyd = gp.multidict({(1,1) : [900, 0.31, 90], (1,2) : [950, 0.33, 95], (1,3): [1000, 0.35, 105], (1,4): [1100, 0.38, 120],
                                 (2,1) : [1400, 0.47, 150], (2,2) : [1500, 0.5, 165], (2,3): [1600, 0.53, 185], (2,4): [1700, 0.56, 210]
                                 })

In [ ]:
# Hhtj
model.remove(H)
indices_hydro = [(h,t,j) for h in hydro for t in periods for j in range(1,5)]   # j is the index of the current palier
H = model.addVars(indices_hydro, vtype=gp.GRB.BINARY,name=[f'H{h}{t}{j}' for h,t,j in indices_hydro])
#demand
for t in periods:
  model.remove(model.getConstrByName(f'Demand_{t}'))
  model.addConstr(gp.quicksum(P[i,t] for i in types) + gp.quicksum(H[h,t,j]*(Phyd[h,j]-vitesse[h,j]*duration[t-1]*Cpompage) for h in hydro for j in range(1,5)) >= load[t-1], name=f'Demand_{t}')
#reserve
  model.remove(model.getConstrByName(f'Reserve_{t}'))
  model.addConstr(gp.quicksum(Y[i,t]*Pmax[i] for i in types) + gp.quicksum((1-H[h,t,j])*Phyd[h,j] + H[h,t,j]*(-vitesse[h,j]*duration[t-1]*Cpompage) for h in hydro for j in range(1,5)) >= (1+r)*load[t-1], name=f'Reserve_{t}')
# bht
  model.remove(model.getConstrByName(f'Activated_Hydro_{h},{t}'))
  for h in hydro:
      model.addConstr(b[h,t] >= gp.quicksum(H[h,t,j] - H[h,prev(t,option),j] for j in range(1,5)), name=f'Activated_Hydro_{h},{t}')

Newobj = gp.quicksum((costs[i]*P[i,t]+ Cbase[i]*Y[i,t] - costs[i]*Pmin[i]*Y[i,t])*duration[t-1]+Cstart[i]*d[i,t] for i,t in indices)
Newobj += gp.quicksum(Chyd[h,j]*H[h,t,j]*duration[t-1] for h,t,j in indices_hydro)
Newobj += gp.quicksum( StartHyd[h]*b[h,t] for h in hydro for t in periods )

model.setObjective(Newobj,gp.GRB.MINIMIZE)
model.update()
model.optimize()